# QMMM workflow using LAMMPS and VOTCA-XTP

## What is this tutorial about
In this tutorial, we will learn how to set and perform excited state calculation using the Votca XTP library. We will use thiophene as our QM region.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
The XTP package offers the following command line interface that the user can interact with:
* [xtp_map](https://votca.github.io/xtp_map.html)
* [xtp_parallel](https://votca.github.io/xtp_parallel.html)
* [xtp_run](https://votca.github.io/xtp_run.html)
* [xtp_tools](https://votca.github.io/xtp_tools.html)

Run the following command to view the help message of `xtp_tools`:

In [1]:
!xtp_tools -h

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_tools, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:18:25)
votca_csg, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:14:08)
votca_tools, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:13:13)

Runs excitation/charge transport tools



Allowed options:
  -h [ --help ]                 display this help and exit
  --verbose                     be loud and noisy
  --verbose1                    be very loud and noisy
  -v [ --verbose2 ]             be extremly loud and noisy
  -o [ --options ] arg          calculator options
  -t [ --nthreads ] arg (=1)    number of threads to create

Tools:
  -e [ --execute ] arg        List of tools separated by ',' or ' '
  -l [ --list ]               Lists all available tools
  -d [ --description ] arg    Short description of a tool
  -n [ --name ] arg           Name of the job to run



### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Setting the environment


Create a folder to store the input `Options` for XTP

In [2]:
!mkdir -p OPTIONFILES

Remove previous hdf5 file

In [3]:
!rm -f state.hdf5

Import an small python module to edit xml files

In [4]:
from xml_editor import add_section, edit_calculator
import os

## Generate the topology from the Gromacs file
We will first generate the mapping from MD coordinates to segments, creating an [hdf5 file](https://www.hdfgroup.org/solutions/hdf5/) to store the results. You can explore the generated `state.hdf5` file with e.g. hdf5itebrowser. In Python, you can use the [h5py library](https://www.h5py.org/). The command to generate the mapping is the following,

In [5]:
!xtp_map -v -t MD_FILES/newfile.data -c MD_FILES/traj.dump -s system.xml -f state.hdf5 -i 99 > mapping.out

##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping. We will need to copy the input from the `VOTCA` installation path to our local copy.

In [6]:
!cp "$VOTCASHARE/xtp/xml/mapchecker.xml" OPTIONFILES/

The `${VOTCASHARE}` environmental variable is set to the path that you provided during the `VOTCA` [installation](https://github.com/votca/votca/blob/master/share/doc/INSTALL.md), by the default is set to `/usr/local/votca`.

We need to update the `mapchecker` options with the file containing the definition of the system. For doing, so we will use the python module `xml_editor` that we have imported above. The first argument is the name of the calculator (i.e. `mapchecker`), the seconds argument is the name of the option to change and the third one, the value to use.

In [7]:
edit_calculator("mapchecker", "map_file", "system.xml")

The option 'map_file' on file 'OPTIONFILES/mapchecker.xml' has been set to 'system.xml'


In the [mapchecker section of the manual](https://votca.github.io/mapchecker.html) you can find a table with the `mapchecker` input variables and their corresponding defaults. Finally, the following command run the check

In [8]:
!xtp_run -e mapchecker -o OPTIONFILES/mapchecker.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_run, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:18:25)
votca_csg, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:14:08)
votca_tools, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:13:13)

Initializing calculator
... mapchecker 
1 frames in statefile, Ids are: 10000 
Starting at frame 10000
Evaluating frame 10000
Import MD Topology (i.e. frame 10000) from state.hdf5
.... 
... mapchecker 
Writing segments to md_segments_step_10000.pdb
Writing qmmolecules to qm_segments_n_step_10000.pdb
Writing polarsegments to mp_segments_e_step_10000.pdb
Writing polarsegments to mp_segments_h_step_10000.pdb

Changes have not been written to state file.


## Neighborlist Calculation
The following step is to determine the neighbouring pairs for exciton transport. We will need first to get a copy of the input

In [9]:
!xtp_run -e neighborlist -o OPTIONFILES/neighborlist.xml -f state.hdf5 -t 4

========   VOTCA (http://www.votca.org)   ========

please submit bugs to bugs@votca.org

xtp_run, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:18:25)
votca_csg, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:14:08)
votca_tools, version 1.7-dev gitid: 6e47b87 (dirty) (compiled Jun  2 2020, 10:13:13)

Initializing calculator
... neighborlist 
1 frames in statefile, Ids are: 10000 
Starting at frame 10000
Evaluating frame 10000
Import MD Topology (i.e. frame 10000) from state.hdf5
.... 
... neighborlist  Using 4 threads
Evaluating 1000 segments for neighborlist. 
 ... ... Evaluating 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

 ... ... Created 90067 direct pairs.
Writing MD topology (step = 10000, time = 0) to state.hdf5
... . 
